In [1]:
# .env 파일을 읽어서 환경변수로 설정
from dotenv import load_dotenv

# 토큰 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("sesac-CH1")

LangSmith 추적을 시작합니다.
[프로젝트명]
sesac-CH1


## 데이터를 효과적으로 전달하는 방법

- `RunnablePassthrough` 는 입력을 변경하지 않거나 추가 키를 더하여 전달할 수 있습니다. 
- `RunnablePassthrough()` 가 단독으로 호출되면, 단순히 입력을 받아 그대로 전달합니다.
- `RunnablePassthrough.assign(...)` 방식으로 호출되면, 입력을 받아 assign 함수에 전달된 추가 인수를 추가합니다.

### RunnablePassthrough


In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template("{num} 의 소인수는?")
llm = ChatOpenAI(temperature=0)

# chain 을 생성합니다.
chain = prompt | llm

chain 을 `invoke()` 하여 실행할 때는 입력 데이터의 타입이 딕셔너리여야 합니다.

In [7]:
# chain 을 실행합니다.
chain.invoke({"num": 66})

AIMessage(content='2, 3, 11', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 16, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C4N78xIKP0QAa0nUTPu188ElnzFez', 'finish_reason': 'stop', 'logprobs': None}, id='run-dab7a6c2-3c98-418f-83f7-60bf17cd2f7f-0', usage_metadata={'input_tokens': 16, 'output_tokens': 7, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

하지만, langchain 라이브러리가 업데이트 되면서 1개의 변수만 템플릿에 포함하고 있다면, 값만 전달하는 것도 가능합니다.

In [8]:
# chain 을 실행합니다.
chain.invoke(15)

AIMessage(content='15의 소인수는 3과 5입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 16, 'total_tokens': 30, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C4N7LIT1W81uHuZSeXZV3mxAyGepm', 'finish_reason': 'stop', 'logprobs': None}, id='run-8faa071b-8ad1-4b81-a52f-8125140755ff-0', usage_metadata={'input_tokens': 16, 'output_tokens': 14, 'total_tokens': 30, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

아래는 `RunnablePassthrough` 를 사용한 예제입니다.

`RunnablePassthrough` 는 `runnable` 객체이며, `runnable` 객체는 `invoke()` 메소드를 사용하여 별도 실행이 가능합니다.


In [9]:
from langchain_core.runnables import RunnablePassthrough

# runnable
RunnablePassthrough().invoke({"num": 10})

{'num': 10}

아래는 `RunnablePassthrough` 로 체인을 구성하는 예제입니다.

In [10]:
runnable_chain = {"num": RunnablePassthrough()} | prompt | ChatOpenAI()

# dict 값이 RunnablePassthrough() 로 변경되었습니다.
runnable_chain.invoke(10)

AIMessage(content='10의 소인수는 2와 5이다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 16, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C4N7WBXexNifN6jGMXbdPMOzpVFK4', 'finish_reason': 'stop', 'logprobs': None}, id='run-a193c396-f359-4206-afe0-a7e3dcce6433-0', usage_metadata={'input_tokens': 16, 'output_tokens': 15, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

다음은 `RunnablePassthrough.assign()` 을 사용하는 경우와 비교한 결과입니다.


In [11]:
RunnablePassthrough().invoke({"num": 1})

{'num': 1}

`RunnablePassthrough.assign()`

- 입력 값으로 들어온 값의 key/value 쌍과 새롭게 할당된 key/value 쌍을 합칩니다.

In [12]:
# 입력 키: num, 할당(assign) 키: new_num
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 1})

{'num': 1, 'new_num': 3}

## RunnableParallel

### 여러 Runnable 인스턴스를 병렬로 실행할 수 있습니다.

In [13]:
from langchain_core.runnables import RunnableParallel

# RunnableParallel 인스턴스를 생성합니다. 이 인스턴스는 여러 Runnable 인스턴스를 병렬로 실행할 수 있습니다.
runnable = RunnableParallel(
    # RunnablePassthrough 인스턴스를 'passed' 키워드 인자로 전달합니다. 이는 입력된 데이터를 그대로 통과시키는 역할을 합니다.
    passed=RunnablePassthrough(),
    # 'extra' 키워드 인자로 RunnablePassthrough.assign을 사용하여, 'mult' 람다 함수를 할당합니다. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값을 3배로 증가시킵니다.
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    # 'modified' 키워드 인자로 람다 함수를 전달합니다. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값에 1을 더합니다.
    modified=lambda x: x["num"] + 1,
)

# runnable 인스턴스에 {'num': 1} 딕셔너리를 입력으로 전달하여 invoke 메소드를 호출합니다.
runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

Chain 도 RunnableParallel 적용할 수 있습니다.


In [14]:
chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 수도는?")
    | ChatOpenAI()
)
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 면적은?")
    | ChatOpenAI()
)

In [15]:
combined_chain = RunnableParallel(capital=chain1, area=chain2)
combined_chain.invoke("대한민국")

{'capital': AIMessage(content='서울특별시입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 19, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C4N8v0R0XYOBI2dmYlU3UTGCZbCbC', 'finish_reason': 'stop', 'logprobs': None}, id='run-6b8ce644-715f-4ba1-b64c-e04547f36080-0', usage_metadata={'input_tokens': 19, 'output_tokens': 10, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 'area': AIMessage(content='대한민국의 총 면적은 약 100,363 km² 입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 20, 'total_tokens': 45, 'completion_tokens_det

## RunnableLambda

RunnableLambda 를 사용하여 사용자 정의 함수를 맵핑할 수 있습니다.


In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime


def get_today(a):
    # 오늘 날짜를 가져오기
    return datetime.today().strftime("%b-%d")


# 오늘 날짜를 출력
get_today(None)

'Aug-14'

In [17]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template(
    "{today} 가 생일인 유명인 {n} 명을 나열하세요. 생년월일을 표기해 주세요."
)
llm = ChatOpenAI(temperature=0, model_name="gpt-4.1-mini")

# chain 을 생성합니다.
chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [19]:
# 출력
print(chain.invoke(10))

다음은 8월 14일이 생일인 유명인 10명과 그들의 생년월일입니다:

1. 스티브 마틴 (Steve Martin) - 1945년 8월 14일  
2. 매기 그레이스 (Maggie Grace) - 1983년 8월 14일  
3. 윌리엄 허트 (William Hurt) - 1950년 8월 14일  
4. 샤를리즈 테론 (Charlize Theron) - 1975년 8월 14일  
5. 데이비드 크로스 (David Cross) - 1964년 8월 14일  
6. 크리스틴 벨 (Kristen Bell) - 1980년 8월 14일  
7. 제임스 캐머런 (James Cameron) - 1954년 8월 14일  
8. 제니퍼 러브 휴잇 (Jennifer Love Hewitt) - 1979년 8월 14일  
9. 샤론 스톤 (Sharon Stone) - 1958년 8월 14일  
10. 타이슨 게이 (Tyson Gay) - 1982년 8월 14일  

필요하시면 더 많은 정보를 제공해 드릴 수 있습니다.


`itemgetter` 를 사용하여 특정 키를 추출합니다.

In [ ]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI


# 문장의 길이를 반환하는 함수입니다.
def length_function(text):
    return len(text)


# 두 문장의 길이를 곱한 값을 반환하는 함수입니다.
def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


# _multiple_length_function 함수를 사용하여 두 문장의 길이를 곱한 값을 반환하는 함수입니다.
def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("{a} + {b} 는 무엇인가요?")
model = ChatOpenAI()

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("word1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("word1"), "text2": itemgetter("word2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

In [ ]:
chain.invoke({"word1": "hello", "word2": "world"})

AIMessage(content='5 + 25는 30입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 22, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C4NAN9l4VjI0kqd5gxfg8QLIts5bI', 'finish_reason': 'stop', 'logprobs': None}, id='run-af1f36e5-f9fd-4066-af95-c767b5dfc27c-0', usage_metadata={'input_tokens': 22, 'output_tokens': 9, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [25]:
# 필요한 라이브러리 임포트
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    RunnablePassthrough,
    RunnableParallel,
    RunnableLambda,
)
from langchain_core.output_parsers import StrOutputParser
import json
import random  # random 라이브러리 추가

# --- (이전 코드와 동일한 부분) ---

# 모델 초기화 (gpt-4o-mini는 빠르고 저렴하여 테스트에 용이합니다)
model = ChatOpenAI(model="gpt-4o-mini")

# 1. 요리의 재료
prompt1 = ChatPromptTemplate.from_template(
    "{food}의 재료를 알려줘"
)
chain1 = {"food": RunnablePassthrough()} | prompt1 | model | StrOutputParser()

# 2. 요리법
prompt2 = ChatPromptTemplate.from_template(
    "{food}의 요리법을 알려줘"
)
chain2 = {"food": RunnablePassthrough()} | prompt2 | model | StrOutputParser()

# 3. 맛집 추천
prompt3 = ChatPromptTemplate.from_template(
    "서울 강동구에서 {food}의 맛집을 추천해줘"
)
chain3 = {"food": RunnablePassthrough()} | prompt3 | model | StrOutputParser()

# 4. RunnableParallel을 사용하여 세개의 체인 통합
combined_chain = RunnableParallel(food_ingredient=chain1, food_cooking=chain2, food_restaurant=chain3)


# 5. 요리의 이름을 랜덤으로 선택하는 함수 정의
def choice_country(_):  # LangChain 체인과 연결하기 위해 입력 인자를 받도록 정의합니다.
    """주어진 리스트에서 국가를 하나 랜덤으로 선택합니다."""
    foods = ["스파게티", "햄버거", "피자", "카레", "김밥"]
    select_food = random.choice(foods)  # 랜덤으로 음식 선택

    # 어떤 나라가 선택되었는지 확인하기 위해 출력
    print(f"랜덤 선택된 요리: {select_food}")

    return select_food


# 5. RunnableLambda로 함수를 체인에 통합하고 최종 체인 완성
# [나라 선택 함수] -> [기존의 병렬 체인] 순서로 실행
final_chain = RunnableLambda(choice_country) | combined_chain

# 6. 최종 체인 실행
# 입력값이 필요 없으므로 None을 전달 (또는 비워둠)
result = final_chain.invoke(None)

# 7. 결과 출력
print("\n--- 최종 결과 ---")
print(json.dumps(result, indent=2, ensure_ascii=False))

🌍 랜덤 선택된 나라: 햄버거

--- 최종 결과 ---
{
  "food_ingredient": "햄버거는 다양한 재료로 구성됩니다. 기본적인 재료는 다음과 같습니다:\n\n1. **번(Bun)**: 햄버거의 빵 부분으로, 일반적으로 위와 아래에 각각 한 개씩 존재합니다.\n2. **패티(Patty)**: 고기 또는 식물 기반 재료로 만들어진 부분입니다. 일반적으로 소고기 패티가 가장 많이 사용되지만, 닭고기, 돼지고기, 생선, 혹은 채식용 버거 등 다양한 옵션이 있습니다.\n3. **치즈(Cheese)**: 보통 햄버거 위에 얹어 녹인 치즈가 사용됩니다. 체다, 모짜렐라, 스위스 치즈 등이 일반적입니다.\n4. **야채(Vegetables)**: 상추, 토마토, 양파, 피클 등 다양한 신선한 야채가 추가됩니다.\n5. **소스(Sauces)**: 마요네즈, 케첩, 머스타드, 바비큐 소스 등 다양한 소스가 사용되어 풍미를 더합니다.\n\n이 외에도 베이컨, 아보카도, 칠리, 다양한 향신료와 허브 등을 추가하여 햄버거를 더욱 풍부하게 만들 수 있습니다.",
  "food_cooking": "햄버거를 만드는 기본적인 요리법을 소개해드릴게요! 간단한 재료와 단계를 통해 맛있는 햄버거를 만들 수 있습니다.\n\n### 재료\n- 다진 소고기 (쇠고기) 500g\n- 햄버거 번 (빵) 4개\n- 소금, 후추 (기호에 따라)\n- 치즈 (체다 치즈 등) 4장 (선택 사항)\n- 토마토 1개 (슬라이스)\n- 양상추 몇 장\n- 양파 (슬라이스, 선택 사항)\n- 마요네즈, 케첩, 머스타드 (선택 사항)\n\n### 요리 방법\n\n1. **패티 만들기**:\n   - 다진 소고기를 큰 그릇에 담고 소금과 후추로 간을 합니다.\n   - 손으로 고기를 적당한 크기로 나누어서 (약 125g 정도) 둥글게 모양을 만듭니다. 그런 다음, 가운데를 약간 눌러서 두께가 고르게 되도록 합니다.\n\n2. **굽기**:\n   - 팬이나 그릴을 중강불로 예열합니다. 팬에 조금의 기름